# Train a totalVI model on CITE-seq and impute expression

In [49]:
import scvi
import scanpy as sc
import anndata as ad
import pandas as pd
import matplotlib.pyplot as plt
import mudata as md

In [ ]:
citeseq_data_dir = '/home/projects/amit/floriani/Lab/PROJECTS/FlowVI/data/raw/CITE_seq/BNHL/'
adata_citeseq_rna = ad.read_h5ad(citeseq_data_dir + '2024-11-06_BNHL_CITEseq_Tcells_RNA.h5ad')
adata_citeseq_prot = ad.read_h5ad(citeseq_data_dir + '2024-12-05_BNHL_CITEseq_Tcells_protein_cleaned_integrated_imputed_arcsinh_BB.h5ad')

In [ ]:
# combine into mudata
mdata = md.MuData({"rna": adata_citeseq_rna, "protein": adata_citeseq_prot})
mdata

/home/projects/amit/floriani/mambaforge/envs/cytovi_gpu/lib/python3.12/site-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/home/projects/amit/floriani/mambaforge/envs/cytovi_gpu/lib/python3.12/site-packages/mudata/_core/mudata.py:931: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(
/home/projects/amit/floriani/mambaforge/envs/cytovi_gpu/lib/python3.12/site-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become

MuData object with n_obs × n_vars = 72721 × 36671
  2 modalities
    rna:	72721 x 36601
      obs:	'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'percent.mt', 'PatientID', 'Run', 'Entity', 'Age', 'Sex', 'Barcode_full', 'Phase', 'RNAclusters_res0.4', 'ADTclusters_res0.4', 'Stage', 'PTCategory', 'PTRegimen', 'PTCycles', 'PTDuration', 'DaysSincePT', 'PTIndication', 'PTResponse', 'FTCategory', 'FTRegimen', 'FTCycles', 'FTDuration', 'DaysUntilFT', 'FTIndication', 'FTResponse', 'Survival', 'DaysFollowup', 'Status', 'Karnofsky', 'Idents_res0.4', 'RNAclusters_res1', 'Idents_res1', 'Idents_res1_full', 'Maturation_M', 'predicted.Maturation.score', 'predicted.Maturation', 'MOFAclusters_res0.5', 'MOFAclusters_res1', 'MAPKscore', 'DZ.a.UPscore', 'DZ.a.DWNscore', 'DZ.b.UPscore', 'DZ.b.DWNscore', 'DZ.c.UPscore', 'DZ.c.DWNscore', 'INT.a.UPscore', 'INT.a.DWNscore', 'INT.b.UPscore', 'INT.b.DWNscore', 'INT.c.UPscore', 'INT.c.DWNscore', 'INT.d.UPscore', 'INT.d.DWNscore', 'INT.e.UPscore', 'INT.e.DWNscore', 'LZ.a.UPscore', 'LZ.a.DWNscore', 'LZ.b.UPscore', 'LZ.b.DWNscore', 'PreM.UPscore', 'PreM.DWNscore', 'PBL.a.UPscore', 'PBL.a.DWNscore', 'PBL.b.UPscore', 'PBL.b.DWNscore', 'predicted.Maturation.bulk', 'predicted.Maturation_course.score', 'predicted.Maturation_course', 'predicted.Maturation_fine.score', 'predicted.Maturation_fine', 'KLR', 'MOFAclusters_clean', 'Malignancy', 'State', 'nCount_scenic', 'nFeature_scenic', 'Relapse', 'DaysDiagnosed', 'CellType', 'Subtype', 'Barcode', 'S.Score', 'G2M.Score', 'seurat_clusters', 'CT_rnaCluster', 'integratedRNA_snn_res.0.4', 'integratedADT_snn_res.0.4', 'integratedRNA_snn_res.1', 'integratedADT_snn_res.0.8', 'integratedADT_snn_res.0.6', 'RNA.weight', 'ADT.weight', 'wsnn_res.0.7', 'wsnn_res.1', 'wsnn_res.1.4', 'IdentI_old', 'integratedADT_snn_res.1', 'integratedADT_snn_res.1.2', 'integratedADT_snn_res.1.4', 'integratedADT_snn_res.1.6', 'Ident_ADT', 'IdentI', 'orig.ident', 'RNA_snn_res.0.4', 'RNA_snn_res.0.5', 'RNA_snn_res.0.3', 'RNA_snn_res.0.2', 'SampleID'
      var:	'highly_variable', 'means', 'dispersions', 'dispersions_norm'
      uns:	'CellType_colors', 'Run_colors', 'hvg', 'log1p', 'neighbors', 'pca', 'umap'
      obsm:	'X_pca', 'X_umap'
      varm:	'PCs'
      layers:	'counts', 'normalized'
      obsp:	'connectivities', 'distances'
    protein:	72721 x 70
      obs:	'nCount_RNA', 'nFeature_RNA', 'nCount_ADT', 'nFeature_ADT', 'percent.mt', 'PatientID', 'Run', 'Entity', 'Age', 'Sex', 'Barcode_full', 'Phase', 'RNAclusters_res0.4', 'ADTclusters_res0.4', 'Stage', 'PTCategory', 'PTRegimen', 'PTCycles', 'PTDuration', 'DaysSincePT', 'PTIndication', 'PTResponse', 'FTCategory', 'FTRegimen', 'FTCycles', 'FTDuration', 'DaysUntilFT', 'FTIndication', 'FTResponse', 'Survival', 'DaysFollowup', 'Status', 'Karnofsky', 'Idents_res0.4', 'RNAclusters_res1', 'Idents_res1', 'Idents_res1_full', 'Maturation_M', 'predicted.Maturation.score', 'predicted.Maturation', 'MOFAclusters_res0.5', 'MOFAclusters_res1', 'MAPKscore', 'DZ.a.UPscore', 'DZ.a.DWNscore', 'DZ.b.UPscore', 'DZ.b.DWNscore', 'DZ.c.UPscore', 'DZ.c.DWNscore', 'INT.a.UPscore', 'INT.a.DWNscore', 'INT.b.UPscore', 'INT.b.DWNscore', 'INT.c.UPscore', 'INT.c.DWNscore', 'INT.d.UPscore', 'INT.d.DWNscore', 'INT.e.UPscore', 'INT.e.DWNscore', 'LZ.a.UPscore', 'LZ.a.DWNscore', 'LZ.b.UPscore', 'LZ.b.DWNscore', 'PreM.UPscore', 'PreM.DWNscore', 'PBL.a.UPscore', 'PBL.a.DWNscore', 'PBL.b.UPscore', 'PBL.b.DWNscore', 'predicted.Maturation.bulk', 'predicted.Maturation_course.score', 'predicted.Maturation_course', 'predicted.Maturation_fine.score', 'predicted.Maturation_fine', 'KLR', 'MOFAclusters_clean', 'Malignancy', 'State', 'nCount_scenic', 'nFeature_scenic', 'Relapse', 'DaysDiagnosed', 'CellType', 'Subtype', 'Barcode', 'S.Score', 'G2M.Score', 'seurat_clusters', 'CT_rnaCluster', 'integratedRNA_snn_res.0.4', 'integratedADT_snn_res.0.4', 'integratedRNA_snn_res.1', 'integratedADT_snn_res.0.8', 'integratedADT_snn_res.0.6', 'RNA.weight', 'ADT.weight', 'wsnn_res.0.7', 'wsnn_res.1

In [ ]:
# compute HVG
sc.pp.highly_variable_genes(
    mdata.mod["rna"],
    n_top_genes=4000,
    flavor="seurat_v3",
    batch_key="Run",
    layer="counts",
)
# Place subsetted counts in a new modality
mdata.mod["rna_subset"] = mdata.mod["rna"][:, mdata.mod["rna"].var["highly_variable"]].copy()

mdata.update()

In [ ]:
# train totalVI model
scvi.model.TOTALVI.setup_mudata(
    mdata,
    rna_layer="counts",
    protein_layer='raw',
    batch_key="Run",
    modalities={
        "rna_layer": "rna_subset",
        "protein_layer": "protein",
        "batch_key": "rna_subset",
    },
)
model = scvi.model.TOTALVI(mdata)
model.train(max_epochs = 50)

model.save('/home/projects/amit/floriani/Lab/PROJECTS/FlowVI/models/BNHL/2025-01-17_CITEseq_TotalVI', overwrite=True)

INFO     Computing empirical prior initialization for protein background.                                          


/home/projects/amit/floriani/mambaforge/envs/cytovi_gpu/lib/python3.12/site-packages/scvi/data/fields/_layer_field.py:115: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  _verify_and_correct_data_format(adata, self.attr_name, self.attr_key)


In [ ]:
# get latent
rna = mdata.mod["rna_subset"]
protein = mdata.mod["protein"]

# arbitrarily store latent in rna modality
TOTALVI_LATENT_KEY = "X_totalVI"
rna.obsm[TOTALVI_LATENT_KEY] = model.get_latent_representation()

In [ ]:
# get batch corrected expression estimates
all_batches = [*adata_citeseq_rna.obs['Run'].drop_duplicates().values]

rna_denoised, protein_denoised = model.get_normalized_expression(
    n_samples=10, return_mean=True, transform_batch=all_batches
)
rna.layers["denoised_rna"] = rna_denoised
protein.layers["denoised_protein"] = protein_denoised

protein.layers["protein_foreground_prob"] = 100 * model.get_protein_foreground_probability(
    n_samples=10, return_mean=True, transform_batch=all_batches
)


mdata.update()

/home/projects/amit/floriani/mambaforge/envs/cytovi_gpu/lib/python3.12/site-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/home/projects/amit/floriani/mambaforge/envs/cytovi_gpu/lib/python3.12/site-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


In [43]:
# save h5ad to disk
rna.write(f'{citeseq_data_dir}2025-01-17_BNHL_CITEseq_combined_RNA_TotalVI_imputed.h5ad')
protein.write(f'{citeseq_data_dir}2025-01-17_BNHL_CITEseq_Tcells_protein_cleaned_integrated_imputed_arcsinh_BB_TotalVI_imputed.h5ad')